In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:

cleaned_comments_df = \
pd.read_csv(os.path.join("static","data","cleaned_comments_239858.csv"),\
    converters={"body_tokens": lambda x: x.strip("[]").replace("'","").split(", "),
               "score": lambda x: int(x)})

In [15]:
cleaned_comments_df.shape

(235579, 6)

In [4]:
cleaned_comments_df["body_tokens_spaced"] = cleaned_comments_df["body_tokens"].apply(lambda x: ' '.join(x))

In [5]:
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(cleaned_comments_df["body_tokens_spaced"])
tf_feature_names = tf_vectorizer.get_feature_names()

In [6]:
n_topics = 249
lda = LatentDirichletAllocation(n_components=n_topics, random_state=4).fit(tf)

In [9]:
import pickle
with open(os.path.join("models", "reddit_LDA_model.pkf"), "wb") as f:
    model = pickle.dump(lda, f)

In [7]:
def display_topics(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic: {topic_idx}")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

n_top_words = 10
display_topics(lda, tf_feature_names, n_top_words)

Topic: 0
question ask history basic developer answer specific seek response possibility
Topic: 1
patch esp realistic stable legendary convenient overhaul career stability bolt
Topic: 2
money cost tax buy pay plastic fund spend return bottle
Topic: 3
cat private owner sims proud hype big volunteer rescue dependent
Topic: 4
kid eat parent fair holy crap tire enough younger peace
Topic: 5
phone fat fell healthy call bridge relative bang iphone appearance
Topic: 6
model paint daughter clue pirate texture delivery plug seat flash
Topic: 7
correct music hop video suggestion artist audio appropriate doe genre
Topic: 8
lol hero trip ridiculous murder manager even leak never bad
Topic: 9
thread topic argue review post question disagree ask direct please
Topic: 10
com imgur jpg png pool duty disk gallery injure bacteria
Topic: 11
must class extremely effort refer proper curse row low begin
Topic: 12
rat catch craft illegal layer coffee instant atleast hawk bio
Topic: 13
support steam account ban

hurt dumb bag trouble alive print anybody doe asian see
Topic: 152
enjoy latest grade beer objective atm wine awhile enjoyable walmart
Topic: 153
update april ship fool 1st til whoever scary entitle bios
Topic: 154
pay bill month payment insurance afford loan smile french need
Topic: 155
kill wave freeze economy perspective priority justify differently doe could
Topic: 156
spot police blind officer chase alliance dust bright horde rage
Topic: 157
amazon walk door respond bed bully neighbor apartment night attitude
Topic: 158
comment answer report delete item helpful repeat earn find see
Topic: 159
dollar intend million era shake belt symbol capitalism popularity pity
Topic: 160
value seriously union protect material genius goat cache mat 100k
Topic: 161
egg australia notification alt border permanent sample equally alternate province
Topic: 162
dude wish heart random could logic best scroll luck good
Topic: 163
lmao mate try yup rich trick frustrate ppl ignorant boom
Topic: 164
sick pa

In [10]:
lda_output = lda.transform(tf)

# column names
topicnames = ["Topic" + str(i) for i in range(n_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(cleaned_comments_df))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [11]:
def color_blue(val):
    color = 'blue' if val > .1 else 'white'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_blue).applymap(make_bold)
df_document_topics

In [ ]:
# module to visualize topics
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='mmds')

In [13]:
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=249, n_jobs=None,
                          perp_tol=0.1, random_state=4, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [14]:
df_document_topic.shape

(235579, 250)

In [25]:
# len(lda_output[0] + lda_output[1])
# len(set(cleaned_comments_df["subreddit"].value_counts))
cleaned_comments_df["subreddit"].value_counts()["totalwar"]

995

In [23]:
subreddit_topic_scores = {}
subreddits = set(cleaned_comments_df["subreddit"])
for subreddit in subreddits:
    subreddit_topic_scores[subreddit] = [0.]*249

In [26]:
for i, comment_topic_scores in enumerate(lda_output):
    subreddit = cleaned_comments_df["subreddit"][i]
    subreddit_topic_scores[subreddit] += comment_topic_scores

KeyboardInterrupt: 